In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI()




In [44]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self, text: str):
    items = text.strip().split(",")
    return list(map(str.strip, items))



In [42]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a comma separated list generate machine. Everything you are asked will be answered with a list of max {max_items}. Do NOT reply anything else"),
  ("ai", "Hi"),
  ("human", "{question}")
])

chain = template | chat | CommaOutputParser()


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']